In [1]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

from core.data_sources import CLOBDataSource
from core.services.timescale_client import TimescaleClient
from decimal import Decimal


clob = CLOBDataSource()
ts_client = TimescaleClient(
    host=os.getenv("TIMESCALE_HOST", "63.250.52.93"),
    port=os.getenv("TIMESCALE_PORT", 5432),
    user=os.getenv("TIMESCALE_USER", "admin"),
    password=os.getenv("TIMESCALE_PASSWORD", "admin"),
    database="timescaledb"
)
await ts_client.connect()

In [2]:
CONNECTOR_NAME = "binance_perpetual"
# Trading Rules Filter
QUOTE_ASSET = "USDT"
MIN_NOTIONAL_SIZE = 5  # In USDT
MAX_PRICE_STEP = 0.001  # Min price step in % (tick size)

trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset(QUOTE_ASSET) \
    .filter_by_min_notional_size(Decimal(MIN_NOTIONAL_SIZE)) \
    .get_all_trading_pairs()
trading_pairs_available = await ts_client.get_available_pairs()
trading_pairs_available = [pair[1] for pair in trading_pairs_available if pair[0] == CONNECTOR_NAME]

trading_pairs = [trading_pair for trading_pair in trading_pairs_available if trading_pair in trading_pairs]

2024-11-07 22:36:03,361 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fafb54c4340>
2024-11-07 22:36:03,362 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7fafb54d5480>, 323.161051594)]']
connector: <aiohttp.connector.TCPConnector object at 0x7fafb54c4310>


In [ ]:
trading_pairs

In [4]:
from core.backtesting.optimizer import StrategyOptimizer


resolution = "1s"
optimizer = StrategyOptimizer(engine="postgres",
                              root_path=root_path,
                              resolution=resolution,
                              db_client=ts_client,
                              db_host="63.250.52.93",
                              db_port=5433,
                              db_user="admin",
                              db_pass="admin",
                              )


In [5]:
optimizer.get_all_study_names()

['xtreet_bb_task_2024-11-07']

In [4]:
from core.backtesting.optimizer import BacktestingConfig, BaseStrategyConfigGenerator
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from controllers.directional_trading.macd_mt_dca import MacdMTDCAControllerConfig
from decimal import Decimal
import optuna

class MACDMTConfigGenerator(BaseStrategyConfigGenerator):
    """
    Strategy configuration generator for MACD MT optimization.
    """
    async def generate_config(self, trial) -> BacktestingConfig:
            
        # paramemeters when dynamic = False
        stop_loss = trial.suggest_float("stop_loss", 0.01, 0.1, step=0.01)
        trailing_stop_activation_price = trial.suggest_float("trailing_stop_activation_price", 0.004, 0.02, step=0.001)
        trailing_delta_ratio = trial.suggest_float("trailing_delta_ratio", 0.05, 0.1, step=0.01)
        trailing_stop_trailing_delta = trailing_stop_activation_price * trailing_delta_ratio
        dca_spread_1 = trial.suggest_float("dca_spread_1", 0.01, 0.04, step=0.01)       
        bb_interval = "1m"
        bb_length = 200
        bb_std = 2

        dynamic_order_spread = False
        dynamic_target =  False



        min_stop_loss=Decimal("0.007")
        max_stop_loss=Decimal("0.1")
        min_trailing_stop=Decimal("0.0025")
        max_trailing_stop=Decimal("0.03")
        
        #bb_interval = trial.suggest_categorical("bb_interval", ["1m", "5m", "15m", "1h"])
        #bb_length = trial.suggest_int("bb_length", 50, 200, step=50)
        #bb_std = trial.suggest_float("bb_std", 1.5, 3, step=0.5)
        #stop_loss = trial.suggest_float("stop_loss", 0.1, 1, step=0.1)
        #dca_spread_1 = trial.suggest_float("dca_spread_1", 0.25, 1, step=0.25)       
        #trailing_stop_activation_price = trial.suggest_float("trailing_stop_activation_price", 0.25, 1, step=0.25)
        #trailing_delta_ratio = trial.suggest_float("trailing_delta_ratio", 0.05, 0.1, step=0.01)
        #trailing_stop_trailing_delta = trailing_stop_activation_price * trailing_delta_ratio

        trailing_stop = TrailingStop(
            activation_price=Decimal(trailing_stop_activation_price),
            trailing_delta=Decimal(trailing_stop_trailing_delta)
        )

        dca_amount_1 = trial.suggest_float("dca_amount_1", 1.0, 3.0, step=1.0)  
        max_executors_per_side = trial.suggest_int("max_executors_per_side", 1, 3, step=1)
        macd_interval_1 = trial.suggest_categorical("macd_interval_1", ["1m", "5m", "15m", "1h"])
        macd_signal_type_1 = trial.suggest_categorical("macd_signal_type_1", ["mean_reversion_1", "mean_reversion_2", "trend_following"])
        macd_fast_1 = trial.suggest_int("macd_fast_1", 9, 59, step=10)
        macd_slow_1 = trial.suggest_int("macd_slow_1", 21, 201, step=10)
        macd_signal_1 = trial.suggest_int("macd_signal_1", 10, 100, step=10)
        macd_number_of_candles_1 = trial.suggest_int("macd_number_of_candles_1", 1, 6, step=2)
        macd_interval_2 = trial.suggest_categorical("macd_interval_2", ["1m", "5m", "15m", "1h"])
        macd_signal_type_2 = trial.suggest_categorical("macd_signal_type_2", ["mean_reversion_1", "mean_reversion_2", "trend_following"])
        macd_fast_2 = trial.suggest_int("macd_fast_2", 9, 59, step=10)
        macd_slow_2 = trial.suggest_int("macd_slow_2", 21, 201, step=10)
        macd_signal_2 = trial.suggest_int("macd_signal_2", 10, 100, step=10)
        macd_number_of_candles_2 = trial.suggest_int("macd_number_of_candles_2", 1, 6, step=2)
        cooldown_time =  trial.suggest_int("cooldown_time", 60*60, 60*60*5, step=60*60)
        take_profit = 100
        time_limit = 60 * 60 * 24 * 2
        total_amount_quote = 100
        executor_refresh_time = 60*2
        
        # Apply logical constraints, pruning if violated
        if (macd_fast_1 >= macd_slow_1 or macd_fast_2 >= macd_slow_2):
            raise optuna.TrialPruned()



        # Create the strategy configuration
        config = MacdMTDCAControllerConfig(
            connector_name="binance_perpetual",
            trading_pair=self.trading_pair,
            candles_trading_pair=self.candles_trading_pair,
            macd_interval_1 = macd_interval_1,
            macd_signal_type_1 = macd_signal_type_1,
            bb_interval = bb_interval,
            bb_length = bb_length,
            bb_std = bb_std,
            dynamic_order_spread = dynamic_order_spread,
            dynamic_target = dynamic_target,
            min_stop_loss = min_stop_loss,
            max_stop_loss = max_stop_loss,
            min_trailing_stop = min_trailing_stop,
            max_trailing_stop = max_trailing_stop,
            macd_fast_1 = macd_fast_1,
            macd_slow_1 = macd_slow_1,
            macd_signal_1 = macd_signal_1,
            macd_number_of_candles_1 = macd_number_of_candles_1,
            macd_interval_2 = macd_interval_2,
            macd_signal_type_2 = macd_signal_type_2,
            macd_fast_2 = macd_fast_2,
            macd_slow_2 = macd_slow_2,
            macd_signal_2 = macd_signal_2,
            macd_number_of_candles_2 = macd_number_of_candles_2,
            dca_amounts = [Decimal("1"), dca_amount_1],
            dca_spreads = [Decimal("-0.00000001"), dca_spread_1],
            total_amount_quote=Decimal(total_amount_quote),
            take_profit=Decimal(take_profit),
            stop_loss=Decimal(stop_loss),
            trailing_stop=trailing_stop,
            time_limit=time_limit,
            max_executors_per_side=max_executors_per_side,
            executor_refresh_time = executor_refresh_time,
            cooldown_time=cooldown_time,
        )

        # Return the configuration encapsulated in BacktestingConfig
        return BacktestingConfig(config=config, start=self.start, end=self.end)

In [ ]:
from core.backtesting.optimizer import StrategyOptimizer
from hummingbot.strategy_v2.backtesting import DirectionalTradingBacktesting

import datetime
import logging



logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

BACKTESTING_DAYS = 7

start_date = datetime.datetime.now() - datetime.timedelta(days=BACKTESTING_DAYS)
end_date = datetime.datetime.now()
logger.info("Generating config generator")

await optimizer._db_client.connect()
for trading_pair in trading_pairs:
    config_generator = MACDMTConfigGenerator(start_date=start_date, end_date=end_date, backtester=DirectionalTradingBacktesting())
    config_generator.trading_pair = trading_pair
    config_generator.candles_trading_pair = trading_pair

    candles = await optimizer._db_client.get_candles(CONNECTOR_NAME, trading_pair,
                                                     resolution, start_date.timestamp(), end_date.timestamp())
    start_time = candles.data["timestamp"].min()
    end_time = candles.data["timestamp"].max()
    config_generator.backtester.backtesting_data_provider.candles_feeds[
        f"{CONNECTOR_NAME}_{trading_pair}_{resolution}"] = candles.data
    config_generator.start = start_time
    config_generator.end = end_time
    await optimizer.optimize(
        study_name="test_final_mt_fixed_pct_1week_oct",
        config_generator=config_generator,
        n_trials=2,
    )

[I 2024-11-07 22:33:27,714] Using an existing study with name 'test_final_mt_fixed_pct_1week_oct' instead of creating a new one.
/home/sapel/miniconda3/envs/quants-lab/lib/python3.10/site-packages/optuna/distributions.py:704: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
2024-11-07 22:33:30,162 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fd482f0d9f0>
2024-11-07 22:33:30,165 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7fd483dc88e0>, 169.961637649)]']
connector: <aiohttp.connector.TCPConnector object at 0x7fd482f0d5a0>
2024-11-07 22:33:36,157 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fd482f0e6b0>
2024-11-07 22:33:36,157 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.Respo

In [ ]:

from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer(root_path=root_path)
optimizer.launch_optuna_dashboard()